In [62]:
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.Linq;
using System.IO;

In [63]:
public class Mark
{
    public string name {get; set;}
    public string group {get; set;}
    public string discipline {get; set;}
    public int mark {get; set;}

    public void PrintInfo()
    {
        Console.WriteLine($"{name}, {group}, {discipline}, {mark}");
    }
}

public class CadetGPA
{
    public string Cadet {get; set;}
    public double GPA {get; set;}

    public void PrintInfo()
    {
        Console.WriteLine($"{Cadet}, {GPA}");
    }
}

Task 3.1 (Highest GPA)

In [68]:
dynamic inputData = JsonConvert.DeserializeObject(File.ReadAllText(@"GetStudentsWithHighestGPA.json"));
List<Mark> marks = JsonConvert.DeserializeObject<List<Mark>>(inputData.data.ToString());

List<CadetGPA> cadetGPAs = marks.GroupBy(m => m.name)
        .Select(g => new CadetGPA
        {
            Cadet = g.Key,
            GPA = g.Average(c => c.mark)
        })
        .ToList();

double maxGPA = cadetGPAs.Max(c => c.GPA);

List<CadetGPA> highestCadetGPAs = cadetGPAs.Where(c => c.GPA == maxGPA).ToList();

var outputData = new
{
    Response = highestCadetGPAs
};

File.WriteAllText(@"GetStudentsWithHighestGPAOutPut.json", JsonConvert.SerializeObject(outputData, Formatting.Indented));



Task 3.2 (GPA by discipine)

In [65]:
dynamic inputData = JsonConvert.DeserializeObject(File.ReadAllText(@"CalculateGPAByDiscipline.json"));
List<Mark> marks = JsonConvert.DeserializeObject<List<Mark>>(inputData.data.ToString());

 var disciplineGPAs = marks.GroupBy(c => c.discipline).Select(g =>
            new JObject(
                new JProperty(g.Key, g.Average(c => c.mark))
            )
        ).ToList();


var outputData = new
{
    Response = disciplineGPAs
};

File.WriteAllText(@"CalculateGPAByDisciplineOutPut.json", JsonConvert.SerializeObject(outputData, Formatting.Indented));


Task 3.3 (GetBestGroupsByDiscipline)

In [66]:
dynamic inputData = JsonConvert.DeserializeObject(File.ReadAllText(@"GetBestGroupsByDiscipline.json"));
List<Mark> marks = JsonConvert.DeserializeObject<List<Mark>>(inputData.data.ToString());

var groupGpas = marks.GroupBy(c => new { c.discipline, c.group })
        .Select(g => new
        {
            discipline = g.Key.discipline,
            group = g.Key.group,
            GPA = g.Average(c => c.mark)
        })
        .ToList();

var bestGroups = groupGpas.GroupBy(g => g.discipline)
    .Select(d => d.OrderByDescending(g => g.GPA).First())
    .ToList();

var outputData = new
{
    Response = bestGroups
};

File.WriteAllText(@"GetBestGroupsByDisciplineOutPut.json", JsonConvert.SerializeObject(outputData, Formatting.Indented));